# DBSCAN

In [1]:
import numpy as np
import pandas as pd
from sklearn.cluster import DBSCAN as skDBSCAN
from cuml import DBSCAN as cumlDBSCAN
import cudf
import os

# Helper Functions

In [2]:
from timeit import default_timer

class Timer(object):
    def __init__(self):
        self._timer = default_timer
    
    def __enter__(self):
        self.start()
        return self

    def __exit__(self, *args):
        self.stop()

    def start(self):
        """Start the timer."""
        self.start = self._timer()

    def stop(self):
        """Stop the timer. Calculate the interval in seconds."""
        self.end = self._timer()
        self.interval = self.end - self.start

In [3]:
import gzip
def load_data(nrows, ncols, cached = 'data/mortgage.npy.gz'):
    if os.path.exists(cached):
        print('use mortgage data')
        with gzip.open(cached) as f:
            X = np.load(f)
        X = X[np.random.randint(0,X.shape[0]-1,nrows),:ncols]
    else:
        print('use random data')
        X = np.random.rand(nrows,ncols)
    df = pd.DataFrame({'fea%d'%i:X[:,i] for i in range(X.shape[1])})
    return df

In [4]:
from sklearn.metrics import mean_squared_error
def array_equal(a,b,threshold=5e-3,with_sign=True):
    a = to_nparray(a)
    b = to_nparray(b)
    if with_sign == False:
        a,b = np.abs(a),np.abs(b)
    res = mean_squared_error(a,b)<threshold
    return res

def to_nparray(x):
    if isinstance(x,np.ndarray) or isinstance(x,pd.DataFrame):
        return np.array(x)
    elif isinstance(x,np.float64):
        return np.array([x])
    elif isinstance(x,cudf.DataFrame) or isinstance(x,cudf.Series):
        return x.to_pandas().values
    return x

# Run tests

In [13]:
%%time
nrows = 5000
ncols = 128

X = load_data(nrows,ncols)
print('data',X.shape)

use random data
data (5000, 128)
CPU times: user 24.3 ms, sys: 9 µs, total: 24.3 ms
Wall time: 22.3 ms


In [6]:
eps = 3
min_samples = 2

In [14]:
%%time
clustering_sk = skDBSCAN(eps = eps, min_samples = min_samples)
clustering_sk.fit(X)

CPU times: user 6.02 s, sys: 2.6 ms, total: 6.02 s
Wall time: 6.02 s


In [15]:
%%time
X_cudf = cudf.DataFrame.from_pandas(X)

CPU times: user 200 ms, sys: 12.2 ms, total: 212 ms
Wall time: 210 ms


In [16]:
%%time
clustering_cuml = cumlDBSCAN(eps = eps, min_samples = min_samples)
clustering_cuml.fit(X_cudf)

CPU times: user 248 ms, sys: 80 µs, total: 248 ms
Wall time: 247 ms


In [17]:
passed = array_equal(clustering_sk.labels_,clustering_cuml.labels_)
message = 'compare dbscan: cuml vs sklearn labels_ %s'%('equal'if passed else 'NOT equal')
print(message)

compare dbscan: cuml vs sklearn labels_ equal


# Run fit_predict on sklearn and cuml DBSCAN

In [19]:
clustering_ft_sk = skDBSCAN(eps = eps, min_samples = min_samples)
result_sk = clustering_ft_sk.fit_predict(X)

In [23]:
clustering_ft_cuml = cumlDBSCAN(eps = eps, min_samples = min_samples)
result_cuml = clustering_ft_cuml.fit_predict(X_cudf)

In [25]:
passed = array_equal(result_cuml,result_sk)
message = 'compare dbscan: cuml vs sklearn labels_ %s'%('equal'if passed else 'NOT equal')
print(message)

compare dbscan: cuml vs sklearn labels_ equal
